In [1]:
from twin import graphmatrix as gm
from twin import embedding as emb
from twin.pipelines import graph_embedding_all
from twin.plots import application_plot
import twin.dataset as dataset

import ipywidgets as widgets
import matplotlib.pyplot as plt
import networkx as nx

2025-02-26 18:47:38.772 | INFO     | twin.config:<module>:11 - PROJ_ROOT path is: /home/alex/Projects/twin


In [2]:
G = dataset.small_world(150, 12, 0.01) # dataset.barabasi_albert(n=500, k=7) # dataset.tree_of_cliques( h=3, k=2, n=4 )# dataset.stick_sculpture(25)

In [3]:
#### %matplotlib inline

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display
import numpy as np

# 1. Define Widgets
title_text = widgets.Text(
    value='My Title',
    placeholder='Type something',
    description='Title:',
    disabled=False
)

slider_alpha = widgets.FloatSlider(
    value=0.85,
    min=0,
    max=1,
    step=0.05,
    description='α:',
    continuous_update=False,
    layout=Layout(width='40%')
)

slider_loglambda = widgets.FloatSlider(
    value=0,
    min=-3,
    max=3,
    step=0.5,
    description='λ (log):',
    continuous_update=False,
    layout=Layout(width='40%')
)

slider_steps = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='steps (k):',
    continuous_update=False,
    layout=Layout(width='40%')
)

text_seed = widgets.BoundedIntText(
    value=0,
    min=0,
    max=1000,
    step=1,
    description='Seed:',
    continuous_update=False,
    layout=Layout(width='40%'),
    disabled=False
)

toggle_3d = widgets.ToggleButton(
    value=False,
    description='3D',
    icon='check',
    layout=Layout(width='10%')
)

toggle_exact = widgets.ToggleButton(
    value=False,
    description='Exact',
    icon='check',
    layout=Layout(width='10%')
)

toggle_normalize = widgets.ToggleButton(
    value=True,
    description='Norm',
    icon='check',
    layout=Layout(width='10%')
)

execute_button = widgets.Button(
    description='Run',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    icon='play'
)

# visualization options
slider_markersize_vertex = widgets.IntSlider( 
    value=30,
    min=1,
    max=100,
    step=1,
    description='V size:',
    style={'description_width': '40px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

slider_markersize_edge = widgets.IntSlider(
    value=18,
    min=1,
    max=100,
    step=1,
    description='E size:',
    style={'description_width': '40px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

toggle_edges = widgets.ToggleButton(
    value=False,
    description='Edges',
    icon='check',
    layout=Layout(width='20%')
)

slider_linewidth = widgets.FloatSlider(
    value=2,
    min=0.1,
    max=10,
    step=0.1,
    description='Edge width:',
    style={'description_width': '80px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

slider_transparency = widgets.FloatSlider(
    value=0.85,
    min=0,
    max=1,
    step=0.05,
    description='Edge transp:',
    style={'description_width': '90px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)


toggle_slopes = widgets.ToggleButton(
    value=False,
    description='Slopes',
    icon='check',
    layout=Layout(width='20%')
)

slope_width = widgets.FloatSlider(
    value=2,
    min=0.1,
    max=10,
    step=0.1,
    description='Slope width:',
    style={'description_width': '80px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

slope_length = widgets.FloatSlider(
    value=1.0,
    min=0.01,
    max=2.0,
    step=0.05,
    description='Slope length:',
    style={'description_width': '80px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

slope_alpha = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='Slope transp:',
    style={'description_width': '90px'},
    readout=False,
    continuous_update=False,
    layout=Layout(width='40%')
)

output_plot = widgets.Output()

# 2. Define labels for each row (or group of widgets)
top_row_label = widgets.Label(value="Main Title & Feature Toggle")
embedding_label = widgets.Label(value="Embedding parameters", layout=Layout(width='20%'))
bottom_row_label = widgets.Label(value="Visualization options")

# We'll store the "previous" expensive parameters here.
# If using a class-based design, you could store this as a class attribute instead.
previous_params = None
experiment = None

# 3. Define the event handler (callback)
def on_execute_button_clicked(b):
    """
    This function will be called whenever the 'Run' button is clicked.
    It can process the values of the widgets and display output in the output area.
    """

    global previous_params
    global experiment

    # 1) Build a dictionary of "key parameters" for expensive computations.
    #    In this case, only 'slider_a' affects heavy computation.
    new_params = {
        'G': G,
        'alpha': slider_alpha.value,
        'loglambda': slider_loglambda.value,
        'steps': slider_steps.value,
        'toggle_3d': toggle_3d.value,
        'run_exact': toggle_exact.value,
        'normalize': toggle_normalize.value,
        'seed': text_seed.value
    }

    # 2) Compare to previous_params
    if new_params == previous_params:
        # Same key parameters => Skip heavy computations
        None
    else:
        # Different => Perform expensive computations (pretend we do something big)
        
        dim_embedding = 3 if new_params['toggle_3d'] else 2
        alpha = new_params['alpha']
        k = new_params['steps']
        lambda_par = 10 ** new_params['loglambda']
        run_exact = new_params['run_exact']
        normalize = new_params['normalize']
        seed = new_params['seed']

        experiment = graph_embedding_all(
            G, # input graph
            twinmatrix_kw={"alpha": alpha, "k": k}, # twin matrix parameters
            #
            # --- select one of the following embedding methods ---
            embedding=emb.SGtSNELayout, embedding_kw={"lambda_par": lambda_par, "run_exact": run_exact},
            # embedding=emb.SpectralLayout, embedding_kw={"normed": True},
            # embedding=emb.SpringLayout, embedding_kw={},
            # 
            # --- the rest are common parameters ---
            seed=seed,
            d=dim_embedding,
            common_init=True,
            normalize=normalize
        )

        previous_params = new_params

    with output_plot:
        # Clear previous outputs (so the figure updates cleanly)
        output_plot.clear_output(wait=True)

        markersize_vertex = slider_markersize_vertex.value
        markersize_edge = slider_markersize_edge.value
        linewidth = slider_linewidth.value
        alpha = slider_transparency.value
        segment_width = slope_width.value
        segment_length = slope_length.value
        show_edges_original = toggle_edges.value
        show_edges_segmenets = toggle_slopes.value
        segment_alpha = slope_alpha.value

        fig = application_plot(
            experiment,
            markersize_vertex=markersize_vertex,
            markersize_edge=markersize_edge,
            linewidth=linewidth,
            alpha=alpha,
            show_edges_original=show_edges_original,
            show_edges_segments=show_edges_segmenets,
            segment_alpha=segment_alpha,
            segment_width=segment_width,
            segment_length=segment_length,
        )
        
        # Show the figure inline
        plt.show()
        

# Assign callback to button
execute_button.on_click(on_execute_button_clicked)

# 4. Organize widgets in a layout with labels
# Row 1: label + text field & toggle

# Row 2: label + two sliders
row_1 = VBox([
    embedding_label,
    HBox([slider_alpha,slider_loglambda, slider_steps, text_seed, toggle_3d, toggle_exact, toggle_normalize],
         layout=Layout(margin='0px 0px 0px -60px',   # top, right, bottom, left
                  padding='0px 0px 0px 0px')),
])

# Row 3: label + run button + output
row_3 = VBox([
    bottom_row_label,
    HBox([slider_markersize_vertex, slider_markersize_edge, toggle_edges, slider_linewidth, slider_transparency]),
    HBox([toggle_slopes, slope_width, slope_length, slope_alpha]),    
    HBox([execute_button])
])

# Final panel layout
panel = VBox([
    row_1,
    row_3,
    output_plot
])


# 5. Display the panel
display(panel);